This notebook pulls OHLCV data from an exchange. At the moment it is configured to pull data for the top 100 coins by marketcap. The code is mostly copied from https://github.com/codeninja/CCXT-Historical-Data. 

In [1]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import os
from pathlib import Path
import os.path as osp
import sys
import csv
import ccxt

In [2]:
# -----------------------------------------------------------------------------

root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(''))))
sys.path.append(root + '/python')

# -----------------------------------------------------------------------------

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise   Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv is None:
            break
        if len(ohlcv)==0:
            break
        if ohlcv[0] is []:
            break
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv


def write_to_csv(filename, exchange, data):
    p = Path("./data_new/raw/", str(exchange))
    p.mkdir(parents=True, exist_ok=True)
    full_path = p / str(filename)
    with Path(full_path).open('w+', newline='') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows(data)

     
        
def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    #exchange = getattr(ccxt, exchange_id)({
    #    'enableRateLimit': True,  # required by the Manual
    #})
    
    exchange_id = 'binance'
    exchange_class = getattr(ccxt, exchange_id)
    exchange = exchange_class({
    #'apiKey': api_key,
    #'secret': api_secret,
    'timeout': 30000,
    'enableRateLimit': True,})
    
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    exchange.enableRateLimit=True
    # fetch all candles
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, exchange, ohlcv)
    if len(ohlcv)!=0:
        print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)

In [44]:
def find_last_date(coin_name):
    root = '../../data/data_new/raw/Binance'
    path = osp.join(root,'{}_USDT'.format(coin_name))
    root = '../../data/data_new/raw/Binance'
    path = osp.join(root,'{}_USDT'.format(coin_name))
    path = osp.join(path,'{}_USDT.csv'.format(coin_name))
    df = pd.read_csv(path,names = ['timestamp','open','high','low','close','volume'])
    if len(df) == 0:
        return '2017-01-0100:00:00'
    else:
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        s = (df['timestamp'].tolist()[-1])
        s = str(s)
        r = ''.join(s.split(' '))
        return r

In [45]:
t = os.listdir('../../data/data_new/raw/Binance')
coins_of_interest = [x.split('_')[0] for x in t]

In [47]:
for c in coins_of_interest:
    print(c,find_last_date(c))

USDN 2017-01-0100:00:00
DOGE 2022-11-0714:23:00
ATOM 2022-11-0714:51:00
CSPR 2017-01-0100:00:00
EOS 2022-11-0715:35:00
QNT 2022-11-0715:16:00
NEO 2022-11-0715:52:00
GRT 2022-11-0715:51:00
NEAR 2022-11-0715:09:00
WAVES 2022-11-0716:24:00
TON 2017-01-0100:00:00
1INCH 2022-11-0716:21:00
DCR 2022-11-0716:22:00
FTM 2022-11-0715:48:00
STX 2022-11-0716:18:00
LUNC 2022-11-0715:21:00
CRV 2022-11-0716:00:00
ETC 2022-11-0714:54:00
XDC 2017-01-0100:00:00
ENJ 2022-11-0716:09:00
NEXO 2022-11-0715:56:00
FTT 2022-11-0715:00:00
ADA 2022-11-0714:26:00
KAVA 2022-11-0716:12:00
DASH 2022-11-0716:01:00
USDP 2017-01-0100:00:00
SOL 2022-11-0714:30:00
SHIB 2022-11-0714:36:00
COMP 2022-11-0716:27:00
OKB 2017-01-0100:00:00
CRO 2017-01-0100:00:00
BSV 2017-01-0100:00:00
LINK 2022-11-0714:48:00
DOT 2022-11-0714:34:00
SNX 2022-11-0715:40:00
BNB 2022-11-0714:13:00
FEI 2017-01-0100:00:00
ZEC 2022-11-0715:43:00
ICP 2022-11-0715:21:00
BCH 2022-11-0715:10:00
RUNE 2022-11-0715:58:00
LTC 2022-11-0714:44:00
BAT 2022-11-0716

In [48]:
#coin_name = 'BTC'
#root = '../../data/data_new/raw/Binance'
#path = osp.join(root,'{}_USDT'.format(coin_name))
#path = osp.join(path,'{}_USDT.csv'.format(coin_name))
#df = pd.read_csv(path,names = ['timestamp','open','high','low','close','volume'])
#
#df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
#s = (df['timestamp'].tolist()[-1])
#
#r = str(s)

In [49]:
#''.join(r.split(' '))

In [50]:
#str(s)

In [53]:
"""Downloads the data of the top 100 coins against USDT. Note that one should be a little vigilant
since sometimes it is obvious that the data for a coin doesnt download fully. In this situation,
one has to redo it for just that coin. For instance, over here ETH/USDT didnt download in its entirety."""

for s in coins_of_interest:
    last_date = find_last_date(s)
    s_full = s + '/USDT'
    dir_name = s + '_USDT'
    scrape_candles_to_csv('{}_latest.csv'.format(dir_name,dir_name), 'binance', 3,s_full,'1m',last_date, 1000)
    
    

1000 DOGE/USDT candles in total from 2023-01-24T23:11:00.000Z to 2023-01-25T15:50:00.000Z
2000 DOGE/USDT candles in total from 2023-01-24T06:31:00.000Z to 2023-01-25T15:50:00.000Z
3000 DOGE/USDT candles in total from 2023-01-23T13:51:00.000Z to 2023-01-25T15:50:00.000Z
4000 DOGE/USDT candles in total from 2023-01-22T21:11:00.000Z to 2023-01-25T15:50:00.000Z
5000 DOGE/USDT candles in total from 2023-01-22T04:31:00.000Z to 2023-01-25T15:50:00.000Z
6000 DOGE/USDT candles in total from 2023-01-21T11:51:00.000Z to 2023-01-25T15:50:00.000Z
7000 DOGE/USDT candles in total from 2023-01-20T19:11:00.000Z to 2023-01-25T15:50:00.000Z
8000 DOGE/USDT candles in total from 2023-01-20T02:31:00.000Z to 2023-01-25T15:50:00.000Z
9000 DOGE/USDT candles in total from 2023-01-19T09:51:00.000Z to 2023-01-25T15:50:00.000Z
10000 DOGE/USDT candles in total from 2023-01-18T17:11:00.000Z to 2023-01-25T15:50:00.000Z
11000 DOGE/USDT candles in total from 2023-01-18T00:31:00.000Z to 2023-01-25T15:50:00.000Z
12000 DO

95000 DOGE/USDT candles in total from 2022-11-20T16:31:00.000Z to 2023-01-25T15:50:00.000Z
96000 DOGE/USDT candles in total from 2022-11-19T23:51:00.000Z to 2023-01-25T15:50:00.000Z
97000 DOGE/USDT candles in total from 2022-11-19T07:11:00.000Z to 2023-01-25T15:50:00.000Z
98000 DOGE/USDT candles in total from 2022-11-18T14:31:00.000Z to 2023-01-25T15:50:00.000Z
99000 DOGE/USDT candles in total from 2022-11-17T21:51:00.000Z to 2023-01-25T15:50:00.000Z
100000 DOGE/USDT candles in total from 2022-11-17T05:11:00.000Z to 2023-01-25T15:50:00.000Z
101000 DOGE/USDT candles in total from 2022-11-16T12:31:00.000Z to 2023-01-25T15:50:00.000Z
102000 DOGE/USDT candles in total from 2022-11-15T19:51:00.000Z to 2023-01-25T15:50:00.000Z
103000 DOGE/USDT candles in total from 2022-11-15T03:11:00.000Z to 2023-01-25T15:50:00.000Z
104000 DOGE/USDT candles in total from 2022-11-14T10:31:00.000Z to 2023-01-25T15:50:00.000Z
105000 DOGE/USDT candles in total from 2022-11-13T17:51:00.000Z to 2023-01-25T15:50:0

72000 ATOM/USDT candles in total from 2022-12-06T15:51:00.000Z to 2023-01-25T15:50:00.000Z
73000 ATOM/USDT candles in total from 2022-12-05T23:11:00.000Z to 2023-01-25T15:50:00.000Z
74000 ATOM/USDT candles in total from 2022-12-05T06:31:00.000Z to 2023-01-25T15:50:00.000Z
75000 ATOM/USDT candles in total from 2022-12-04T13:51:00.000Z to 2023-01-25T15:50:00.000Z
76000 ATOM/USDT candles in total from 2022-12-03T21:11:00.000Z to 2023-01-25T15:50:00.000Z
77000 ATOM/USDT candles in total from 2022-12-03T04:31:00.000Z to 2023-01-25T15:50:00.000Z
78000 ATOM/USDT candles in total from 2022-12-02T11:51:00.000Z to 2023-01-25T15:50:00.000Z
79000 ATOM/USDT candles in total from 2022-12-01T19:11:00.000Z to 2023-01-25T15:50:00.000Z
80000 ATOM/USDT candles in total from 2022-12-01T02:31:00.000Z to 2023-01-25T15:50:00.000Z
81000 ATOM/USDT candles in total from 2022-11-30T09:51:00.000Z to 2023-01-25T15:50:00.000Z
82000 ATOM/USDT candles in total from 2022-11-29T17:11:00.000Z to 2023-01-25T15:50:00.000Z

48000 EOS/USDT candles in total from 2022-12-23T07:52:00.000Z to 2023-01-25T15:51:00.000Z
49000 EOS/USDT candles in total from 2022-12-22T15:12:00.000Z to 2023-01-25T15:51:00.000Z
50000 EOS/USDT candles in total from 2022-12-21T22:32:00.000Z to 2023-01-25T15:51:00.000Z
51000 EOS/USDT candles in total from 2022-12-21T05:52:00.000Z to 2023-01-25T15:51:00.000Z
52000 EOS/USDT candles in total from 2022-12-20T13:12:00.000Z to 2023-01-25T15:51:00.000Z
53000 EOS/USDT candles in total from 2022-12-19T20:32:00.000Z to 2023-01-25T15:51:00.000Z
54000 EOS/USDT candles in total from 2022-12-19T03:52:00.000Z to 2023-01-25T15:51:00.000Z
55000 EOS/USDT candles in total from 2022-12-18T11:12:00.000Z to 2023-01-25T15:51:00.000Z
56000 EOS/USDT candles in total from 2022-12-17T18:32:00.000Z to 2023-01-25T15:51:00.000Z
57000 EOS/USDT candles in total from 2022-12-17T01:52:00.000Z to 2023-01-25T15:51:00.000Z
58000 EOS/USDT candles in total from 2022-12-16T09:12:00.000Z to 2023-01-25T15:51:00.000Z
59000 EOS/

25000 QNT/USDT candles in total from 2023-01-08T07:12:00.000Z to 2023-01-25T15:51:00.000Z
26000 QNT/USDT candles in total from 2023-01-07T14:32:00.000Z to 2023-01-25T15:51:00.000Z
27000 QNT/USDT candles in total from 2023-01-06T21:52:00.000Z to 2023-01-25T15:51:00.000Z
28000 QNT/USDT candles in total from 2023-01-06T05:12:00.000Z to 2023-01-25T15:51:00.000Z
29000 QNT/USDT candles in total from 2023-01-05T12:32:00.000Z to 2023-01-25T15:51:00.000Z
30000 QNT/USDT candles in total from 2023-01-04T19:52:00.000Z to 2023-01-25T15:51:00.000Z
31000 QNT/USDT candles in total from 2023-01-04T03:12:00.000Z to 2023-01-25T15:51:00.000Z
32000 QNT/USDT candles in total from 2023-01-03T10:32:00.000Z to 2023-01-25T15:51:00.000Z
33000 QNT/USDT candles in total from 2023-01-02T17:52:00.000Z to 2023-01-25T15:51:00.000Z
34000 QNT/USDT candles in total from 2023-01-02T01:12:00.000Z to 2023-01-25T15:51:00.000Z
35000 QNT/USDT candles in total from 2023-01-01T08:32:00.000Z to 2023-01-25T15:51:00.000Z
36000 QNT/

...

19000 CAKE/USDT candles in total from 2023-01-12T11:22:00.000Z to 2023-01-25T16:01:00.000Z
20000 CAKE/USDT candles in total from 2023-01-11T18:42:00.000Z to 2023-01-25T16:01:00.000Z
21000 CAKE/USDT candles in total from 2023-01-11T02:02:00.000Z to 2023-01-25T16:01:00.000Z
22000 CAKE/USDT candles in total from 2023-01-10T09:22:00.000Z to 2023-01-25T16:01:00.000Z
23000 CAKE/USDT candles in total from 2023-01-09T16:42:00.000Z to 2023-01-25T16:01:00.000Z
24000 CAKE/USDT candles in total from 2023-01-09T00:02:00.000Z to 2023-01-25T16:01:00.000Z
25000 CAKE/USDT candles in total from 2023-01-08T07:22:00.000Z to 2023-01-25T16:01:00.000Z
26000 CAKE/USDT candles in total from 2023-01-07T14:42:00.000Z to 2023-01-25T16:01:00.000Z
27000 CAKE/USDT candles in total from 2023-01-06T22:02:00.000Z to 2023-01-25T16:01:00.000Z
28000 CAKE/USDT candles in total from 2023-01-06T05:22:00.000Z to 2023-01-25T16:01:00.000Z
29000 CAKE/USDT candles in total from 2023-01-05T12:42:00.000Z to 2023-01-25T16:01:00.000Z

110000 CAKE/USDT candles in total from 2022-11-10T06:42:00.000Z to 2023-01-25T16:01:00.000Z
111000 CAKE/USDT candles in total from 2022-11-09T14:02:00.000Z to 2023-01-25T16:01:00.000Z
112000 CAKE/USDT candles in total from 2022-11-08T21:22:00.000Z to 2023-01-25T16:01:00.000Z
113000 CAKE/USDT candles in total from 2022-11-08T04:42:00.000Z to 2023-01-25T16:01:00.000Z
114000 CAKE/USDT candles in total from 2022-11-07T12:02:00.000Z to 2023-01-25T16:01:00.000Z
Saved 114000 candles from 2022-11-07T12:02:00.000Z to 2023-01-25T16:01:00.000Z to CAKE_USDT_latest.csv
1000 FLOW/USDT candles in total from 2023-01-24T23:22:00.000Z to 2023-01-25T16:01:00.000Z
2000 FLOW/USDT candles in total from 2023-01-24T06:42:00.000Z to 2023-01-25T16:01:00.000Z
3000 FLOW/USDT candles in total from 2023-01-23T14:02:00.000Z to 2023-01-25T16:01:00.000Z
4000 FLOW/USDT candles in total from 2023-01-22T21:22:00.000Z to 2023-01-25T16:01:00.000Z
5000 FLOW/USDT candles in total from 2023-01-22T04:42:00.000Z to 2023-01-25T1

87000 FLOW/USDT candles in total from 2022-11-26T06:02:00.000Z to 2023-01-25T16:01:00.000Z
88000 FLOW/USDT candles in total from 2022-11-25T13:22:00.000Z to 2023-01-25T16:01:00.000Z
89000 FLOW/USDT candles in total from 2022-11-24T20:42:00.000Z to 2023-01-25T16:01:00.000Z
90000 FLOW/USDT candles in total from 2022-11-24T04:02:00.000Z to 2023-01-25T16:01:00.000Z
91000 FLOW/USDT candles in total from 2022-11-23T11:22:00.000Z to 2023-01-25T16:01:00.000Z
92000 FLOW/USDT candles in total from 2022-11-22T18:42:00.000Z to 2023-01-25T16:01:00.000Z
93000 FLOW/USDT candles in total from 2022-11-22T02:02:00.000Z to 2023-01-25T16:01:00.000Z
94000 FLOW/USDT candles in total from 2022-11-21T09:22:00.000Z to 2023-01-25T16:01:00.000Z
95000 FLOW/USDT candles in total from 2022-11-20T16:42:00.000Z to 2023-01-25T16:01:00.000Z
96000 FLOW/USDT candles in total from 2022-11-20T00:02:00.000Z to 2023-01-25T16:01:00.000Z
97000 FLOW/USDT candles in total from 2022-11-19T07:22:00.000Z to 2023-01-25T16:01:00.000Z

62000 THETA/USDT candles in total from 2022-12-13T14:42:00.000Z to 2023-01-25T16:01:00.000Z
63000 THETA/USDT candles in total from 2022-12-12T22:02:00.000Z to 2023-01-25T16:01:00.000Z
64000 THETA/USDT candles in total from 2022-12-12T05:22:00.000Z to 2023-01-25T16:01:00.000Z
65000 THETA/USDT candles in total from 2022-12-11T12:42:00.000Z to 2023-01-25T16:01:00.000Z
66000 THETA/USDT candles in total from 2022-12-10T20:02:00.000Z to 2023-01-25T16:01:00.000Z
67000 THETA/USDT candles in total from 2022-12-10T03:22:00.000Z to 2023-01-25T16:01:00.000Z
68000 THETA/USDT candles in total from 2022-12-09T10:42:00.000Z to 2023-01-25T16:01:00.000Z
69000 THETA/USDT candles in total from 2022-12-08T18:02:00.000Z to 2023-01-25T16:01:00.000Z
70000 THETA/USDT candles in total from 2022-12-08T01:22:00.000Z to 2023-01-25T16:01:00.000Z
71000 THETA/USDT candles in total from 2022-12-07T08:42:00.000Z to 2023-01-25T16:01:00.000Z
72000 THETA/USDT candles in total from 2022-12-06T16:02:00.000Z to 2023-01-25T16

38000 XEC/USDT candles in total from 2022-12-30T06:42:00.000Z to 2023-01-25T16:01:00.000Z
39000 XEC/USDT candles in total from 2022-12-29T14:02:00.000Z to 2023-01-25T16:01:00.000Z
40000 XEC/USDT candles in total from 2022-12-28T21:22:00.000Z to 2023-01-25T16:01:00.000Z
41000 XEC/USDT candles in total from 2022-12-28T04:42:00.000Z to 2023-01-25T16:01:00.000Z
42000 XEC/USDT candles in total from 2022-12-27T12:02:00.000Z to 2023-01-25T16:01:00.000Z
43000 XEC/USDT candles in total from 2022-12-26T19:22:00.000Z to 2023-01-25T16:01:00.000Z
44000 XEC/USDT candles in total from 2022-12-26T02:42:00.000Z to 2023-01-25T16:01:00.000Z
45000 XEC/USDT candles in total from 2022-12-25T10:02:00.000Z to 2023-01-25T16:01:00.000Z
46000 XEC/USDT candles in total from 2022-12-24T17:22:00.000Z to 2023-01-25T16:01:00.000Z
47000 XEC/USDT candles in total from 2022-12-24T00:42:00.000Z to 2023-01-25T16:01:00.000Z
48000 XEC/USDT candles in total from 2022-12-23T08:02:00.000Z to 2023-01-25T16:01:00.000Z
49000 XEC/